In [21]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread, pprint
import pandas as pd
import numpy as np
from datetime import datetime

In [22]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('bplinebot-3ccea59ad6d6.json', scope)
client = gspread.authorize(creds)
pp = pprint.PrettyPrinter()
def get_client():
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('bplinebot-3ccea59ad6d6.json', scope)
    client = gspread.authorize(creds)
    return client

In [23]:
#sheet = client.open('AbbokIncomeAssesmentV03').get_worksheet(2)
spread = client.open('userCheckin')

In [24]:
sheet = spread.worksheet('log')

In [25]:
list_of_lists = sheet.get_all_values()

In [26]:
df = pd.DataFrame(list_of_lists[1:], columns = list_of_lists[0])
df = df[pd.to_datetime(df['DATE']).dt.month == datetime.today().month]#datetime.today().month
df.head(10)

,ID,DATE,TIME,USER_ID,TYPE
438,439,2019/09/02,09:29 AM,2,1
439,440,2019/09/02,09:41 AM,3,1
440,441,2019/09/02,09:42 AM,7,1
441,442,2019/09/02,10:24 AM,8,1
442,443,2019/09/02,12:20 PM,1,1
443,444,2019/09/02,12:22 PM,4,1
444,445,2019/09/02,07:18 PM,2,0
445,446,2019/09/02,07:30 PM,8,0
446,447,2019/09/02,07:45 PM,1,0
447,448,2019/09/02,07:45 PM,4,0


In [27]:
df['TIME'] = pd.to_datetime(df['TIME']).values.astype(np.int64)
df['DATE'] = pd.to_datetime(df['DATE']).dt.date
df['USER_ID'] = df['USER_ID'].astype(int)
df['TYPE'] = df['TYPE'].astype(int)
df = df.set_index('ID')
df.shape

(101, 4)

In [28]:
df2 = pd.DataFrame(df.groupby(['USER_ID','DATE','TYPE'])['TIME'].mean())

In [29]:
df2['INOUT'] = pd.to_datetime(df2['TIME']).dt.time
df2

TIME     INOUT
USER_ID DATE       TYPE                               
1       2019-09-02 0     1568144700000000000  19:45:00
                   1     1568118000000000000  12:20:00
        2019-09-03 0     1568144100000000000  19:35:00
                   1     1568122080000000000  13:28:00
        2019-09-04 0     1568144400000000000  19:40:00
                   1     1568120340000000000  12:59:00
        2019-09-05 0     1568149880000000000  21:11:20
                   1     1568142900000000000  19:15:00
        2019-09-06 0     1568130160000000000  15:42:40
                   1     1568123100000000000  13:45:00
        2019-09-09 0     1568144280000000000  19:38:00
                   1     1568116110000000000  11:48:30
        2019-09-10 1     1568122500000000000  13:35:00
2       2019-09-02 0     1568143080000000000  19:18:00
                   1     1568107740000000000  09:29:00
        2019-09-03 0     1568131200000000000  16:00:00
                   1     1568107380000000000  09:23:00
        2019-09-04 1     1568108160000000000  09:36:00
        2019-09-05 0     1568106120000000000  09:02:00
                   1     1568111280000000000  10:28:00
        2019-09-06 0     1568133540000000000  16:39:00
                   1     1568113320000000000  11:02:00
        2019-09-09 0     1568147280000000000  20:28:00
                   1     1568105520000000000  08:52:00
        2019-09-10 1     1568105940000000000  08:59:00
3       2019-09-02 0     1568157660000000000  23:21:00
                   1     1568108460000000000  09:41:00
        2019-09-03 0     1568146080000000000  20:08:00
                   1     1568120040000000000  12:54:00
        2019-09-04 1     1568109780000000000  10:03:00
...                                      ...       ...
7       2019-09-05 1     1568114100000000000  11:15:00
        2019-09-06 0     1568145240000000000  19:54:00
                   1     1568107440000000000  09:24:00
        2019-09-09 0     1568143020000000000  19:17:00
                   1     1568108400000000000  09:40:00
        2019-09-10 1     1568108940000000000  09:49:00
8       2019-09-02 0     1568143800000000000  19:30:00
                   1     1568111040000000000  10:24:00
        2019-09-03 0     1568144580000000000  19:43:00
                   1     1568107980000000000  09:33:00
        2019-09-04 0     1568143920000000000  19:32:00
                   1     1568111700000000000  10:35:00
        2019-09-05 0     1568144280000000000  19:38:00
                   1     1568109720000000000  10:02:00
        2019-09-06 0     1568138880000000000  18:08:00
                   1     1568108820000000000  09:47:00
        2019-09-09 0     1568143020000000000  19:17:00
                   1     1568110560000000000  10:16:00
        2019-09-10 1     1568108820000000000  09:47:00
9       2019-09-03 0     1568144160000000000  19:36:00
                   1     1568107800000000000  09:30:00
        2019-09-04 0     1568146560000000000  20:16:00
                   1     1568107440000000000  09:24:00
        2019-09-05 0     1568149440000000000  21:04:00
                   1     1568107080000000000  09:18:00
        2019-09-06 0     1568148780000000000  20:53:00
                   1     1568118660000000000  12:31:00
        2019-09-09 0     1568145360000000000  19:56:00
                   1     1568105700000000000  08:55:00
        2019-09-10 1     1568107500000000000  09:25:00

[81 rows x 2 columns]

In [30]:
df2 = df2.reset_index()

In [63]:
w_out = df2[df2['TYPE'] == 0].reset_index().drop(["index","TYPE"], axis=1)
w_in = df2[df2['TYPE'] == 1].reset_index().drop(["index","TYPE"], axis=1)

In [64]:
w_out.head(8)

,USER_ID,DATE,TIME,INOUT
0,1,2019-09-02,1568144700000000000,19:45:00
1,1,2019-09-03,1568144100000000000,19:35:00
2,1,2019-09-04,1568144400000000000,19:40:00
3,1,2019-09-05,1568149880000000000,21:11:20
4,1,2019-09-06,1568130160000000000,15:42:40
5,1,2019-09-09,1568144280000000000,19:38:00
6,2,2019-09-02,1568143080000000000,19:18:00
7,2,2019-09-03,1568131200000000000,16:00:00


In [65]:
w_in.head(8)

,USER_ID,DATE,TIME,INOUT
0,1,2019-09-02,1568118000000000000,12:20:00
1,1,2019-09-03,1568122080000000000,13:28:00
2,1,2019-09-04,1568120340000000000,12:59:00
3,1,2019-09-05,1568142900000000000,19:15:00
4,1,2019-09-06,1568123100000000000,13:45:00
5,1,2019-09-09,1568116110000000000,11:48:30
6,1,2019-09-10,1568122500000000000,13:35:00
7,2,2019-09-02,1568107740000000000,09:29:00


In [66]:
new_df = pd.merge(w_out, w_in,  how='left', left_on=['USER_ID','DATE'], right_on = ['USER_ID','DATE'])
new_df

,USER_ID,DATE,TIME_x,INOUT_x,TIME_y,INOUT_y
0,1,2019-09-02,1568144700000000000,19:45:00,1568118000000000000,12:20:00
1,1,2019-09-03,1568144100000000000,19:35:00,1568122080000000000,13:28:00
2,1,2019-09-04,1568144400000000000,19:40:00,1568120340000000000,12:59:00
3,1,2019-09-05,1568149880000000000,21:11:20,1568142900000000000,19:15:00
4,1,2019-09-06,1568130160000000000,15:42:40,1568123100000000000,13:45:00
5,1,2019-09-09,1568144280000000000,19:38:00,1568116110000000000,11:48:30
6,2,2019-09-02,1568143080000000000,19:18:00,1568107740000000000,09:29:00
7,2,2019-09-03,1568131200000000000,16:00:00,1568107380000000000,09:23:00
8,2,2019-09-05,1568106120000000000,09:02:00,1568111280000000000,10:28:00
9,2,2019-09-06,1568133540000000000,16:39:00,1568113320000000000,11:02:00


In [67]:
w_total = w_out.copy()
w_total['TIME'] = new_df['TIME_x'] - new_df['TIME_y']
w_total.head(5)

,USER_ID,DATE,TIME,INOUT
0,1,2019-09-02,26700000000000,19:45:00
1,1,2019-09-03,22020000000000,19:35:00
2,1,2019-09-04,24060000000000,19:40:00
3,1,2019-09-05,6980000000000,21:11:20
4,1,2019-09-06,7060000000000,15:42:40


In [68]:
w_total = w_total.groupby(['USER_ID']).mean()

In [69]:
w_total['TIME'] = pd.to_datetime(w_total['TIME']).dt.time

In [71]:
w_total = w_total.reset_index()
w_total

,USER_ID,TIME
0,1,05:19:25
1,2,06:26:36
2,3,04:03:53
3,4,09:55:45
4,7,09:08:00
5,8,09:11:50
6,9,10:25:24


In [72]:
w_total[w_total['USER_ID'] == 1]['TIME'].values[0].strftime("%H:%M:%S")

'05:19:25'